In [1]:
import pandas as pd 
import numpy as np
import os
data_folder = 'aug-2025-release'

In [9]:
clinical_evidence = os.path.join(data_folder, '01-Aug-2025-ClinicalEvidenceSummaries.tsv')
clinical_evidence = pd.read_csv(clinical_evidence, sep='\t')
clinical_evidence = clinical_evidence[clinical_evidence.therapies.notna()]
clinical_evidence.columns

Index(['molecular_profile', 'molecular_profile_id', 'disease', 'doid',
       'phenotypes', 'therapies', 'therapy_interaction_type', 'evidence_type',
       'evidence_direction', 'evidence_level', 'significance',
       'evidence_statement', 'citation_id', 'source_type', 'asco_abstract_id',
       'citation', 'nct_ids', 'rating', 'evidence_status', 'evidence_id',
       'variant_origin', 'last_review_date', 'evidence_civic_url',
       'molecular_profile_civic_url', 'is_flagged'],
      dtype='object')

In [10]:
clinical_evidence[clinical_evidence.evidence_level == 'A']

,molecular_profile,molecular_profile_id,disease,doid,phenotypes,therapies,therapy_interaction_type,evidence_type,evidence_direction,evidence_level,...,citation,nct_ids,rating,evidence_status,evidence_id,variant_origin,last_review_date,evidence_civic_url,molecular_profile_civic_url,is_flagged
206,EGFR T790M,34,Lung Non-small Cell Carcinoma,3908.0,NaN,Erlotinib,NaN,Predictive,Supports,A,...,"Denis et al., 2015",NaN,5.0,accepted,238,Somatic,2023-01-09 21:46:26 UTC,https://civicdb.org/links/evidence_items/238,https://civicdb.org/links/molecular_profiles/34,False
221,BCR::ABL1 Fusion,1,Acute Myeloid Leukemia,9119.0,NaN,Imatinib,NaN,Predictive,Supports,A,...,"Nimmanapalli et al., 2002",NaN,5.0,accepted,259,Somatic,2023-01-09 21:46:24 UTC,https://civicdb.org/links/evidence_items/259,https://civicdb.org/links/molecular_profiles/1,False
222,BCR::ABL1 Fusion,1,Chronic Myeloid Leukemia,8552.0,NaN,Imatinib,NaN,Predictive,Supports,A,...,"An et al., 2010",NaN,5.0,accepted,260,Somatic,2023-01-09 21:46:24 UTC,https://civicdb.org/links/evidence_items/260,https://civicdb.org/links/molecular_profiles/1,False
223,BCR::ABL1 Fusion,1,Chronic Myeloid Leukemia,8552.0,NaN,"Dasatinib,Nilotinib",Substitutes,Predictive,Supports,A,...,"An et al., 2010",NaN,4.0,accepted,261,Somatic,2023-01-09 21:46:24 UTC,https://civicdb.org/links/evidence_items/261,https://civicdb.org/links/molecular_profiles/1,False
267,MGMT Promoter Methylation,85,Glioblastoma,3068.0,NaN,Temozolomide,NaN,Predictive,Supports,A,...,"Hegi et al., 2005",NaN,5.0,accepted,307,Somatic,2023-01-09 21:46:27 UTC,https://civicdb.org/links/evidence_items/307,https://civicdb.org/links/molecular_profiles/85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487,BRAF V600E,12,Solid Tumor,NaN,NaN,"Dabrafenib,Trametinib",Combination,Predictive,Supports,A,...,"Subbiah et al., 2023",NaN,4.0,accepted,12161,Somatic,2025-01-24 17:19:02 UTC,https://civicdb.org/links/evidence_items/12161,https://civicdb.org/links/molecular_profiles/12,False
4488,BRAF V600E,12,Low Grade Glioma,80829.0,NaN,"Dabrafenib,Trametinib",Combination,Predictive,Supports,A,...,"Bouffet et al., 2023",NaN,4.0,accepted,12162,Somatic,2025-01-13 16:55:37 UTC,https://civicdb.org/links/evidence_items/12162,https://civicdb.org/links/molecular_profiles/12,False
4507,v::NRG1 Fusion,2162,Lung Non-small Cell Carcinoma,3908.0,NaN,Zenocutuzumab-zbco,NaN,Predictive,Supports,A,...,"Schram et al., 2025",NaN,4.0,accepted,12246,Somatic,2025-02-10 18:42:31 UTC,https://civicdb.org/links/evidence_items/12246,https://civicdb.org/links/molecular_profiles/2162,False
4508,v::NRG1 Fusion,2162,Pancreatic Adenocarcinoma,4074.0,NaN,Zenocutuzumab-zbco,NaN,Predictive,Supports,A,...,"Schram et al., 2025",NaN,4.0,accepted,12247,Somatic,2025-02-10 18:45:42 UTC,https://civicdb.org/links/evidence_items/12247,https://civicdb.org/links/molecular_profiles/2162,False
